In [25]:
from cube import Cube, load_model, device
import numpy as np
import random
import torch
import matplotlib.pyplot as plt
import optimal.solver as sv

"""
                  |************|
                  |*0 **1 **2 *|
                  |************|
                  |*3 **4 **5 *|
                  |************|
                  |*6 **7 **8 *|
                  |************|
     |************|************|************|************|
     |*36**37**38*|*18**19**20*|*9 **10**11*|*45**46**47*|
     |************|************|************|************|
     |*39**40**41*|*21**22**23*|*12**13**14*|*48**49**50*|
     |************|************|************|************|
     |*42**43**44*|*24**25**26*|*15**16**17*|*51**52**53*|
     |************|************|************|************|
                  |************|
                  |*27**28**29*|
                  |************|
                  |*30**31**32*|
                  |************|
                  |*33**34**35*|
                  |************|
                  
U1, U2, U3, U4, U5, U6, U7, U8, U9, 
R1, R2, R3, R4, R5, R6, R7, R8, R9, 
F1, F2, F3, F4, F5, F6, F7, F8, F9, 
D1, D2, D3, D4, D5, D6, D7, D8, D9, 
L1, L2, L3, L4, L5, L6, L7, L8, L9, 
B1, B2, B3, B4, B5, B6, B7, B8, B9              
                
    Indices of state (each starting with 9*(n-1)):

                      |  2  5  8 |
                      |  1  4  7 |
                      |  0  3  6 |
             --------------------------------------------
             20 23 26 | 47 50 53 | 29 32 35 | 38 41 44
             19 22 25 | 46 49 52 | 28 31 34 | 37 40 43
             18 21 24 | 45 48 51 | 27 30 33 | 36 39 42
             --------------------------------------------           
                      | 11 14 17 |
                      | 10 13 16 |
                      | 9  12 15 |  

"""

state_conversion = {
    0: 2, 1: 5, 2: 8, 3: 1, 4: 4, 5: 7, 6: 0, 7: 3, 8: 6,
    9: 29, 10: 32, 11: 35, 12: 28, 13: 31, 14: 34, 15: 27, 16: 30, 17: 33,
    18: 47, 19: 50, 20: 53, 21: 46, 22: 49, 23: 52, 24: 45, 25: 48, 26: 51,
    27: 11, 28: 14, 29: 17, 30: 10, 31: 13, 32: 16, 33: 9, 34: 12, 35: 15,
    36: 20, 37: 23, 38: 26, 39: 19, 40: 22, 41: 25, 42: 18, 43: 21, 44: 24,
    45: 38, 46: 41, 47: 44, 48: 37, 49: 40, 50: 43, 51: 36, 52: 39, 53: 42
}

number_to_face = {
    0 : 'U', 1 : 'D', 2 : 'L', 3 : 'R', 4 : 'B', 5 : 'F'
}

nnet = load_model()

def compute_fitness(states):
    # Convert list of states to a NumPy array (if not already an array)
    states_np = np.array(states)
    # Convert NumPy array to a PyTorch tensor
    input_tensor = torch.tensor(states_np, dtype=torch.float32).to(device)
    # Compute output in a single batch
    outputs = nnet(input_tensor)
    fitness_scores = outputs.detach().cpu().numpy()
    torch.cuda.empty_cache()  # Clear CUDA cache here
    return fitness_scores


def test_admissible_heuristic(error):
    
    for j in range(100):
        print(f"Test {j + 1}")
        
        batch = []
        batch_info = []
        
        for i in range(38000):
            cube = Cube()
            random_moves = random.randint(1, 30)
            scramble_str = cube.randomize_n(random_moves)
            batch.append(cube.state)
            batch_info.append((cube.state, random_moves, scramble_str))
            
        print("Batch Completed")

        fitness_scores = compute_fitness(batch)
        
        print("Fitness Computed")
        
        for (cube_state, random_moves, scramble_str), fitness in zip(batch_info, fitness_scores):
            cube = Cube()
            cube.from_state(cube_state)
            heuristic = fitness
            test_val = 1.5 * random_moves
        
            error.append(abs(heuristic - test_val))
            
        print(f"Test {i + 1} finished")
        
    return error
    
def plot_error(error):
    # create box plot
    
    fig = plt.figure(figsize =(10, 7))
    plt.boxplot(error)
    plt.show()
    
    # save box plot
    fig.savefig("admissible_heuristic_error.png")
    
    # create histogram
    hist_fig = plt.hist(error, bins=100)
    plt.show()
    # save histogram
    hist_fig.savefig("admissible_heuristic_error_hist.png")
    
    
    
test_cube = Cube()
test_cube.randomize_n(30)

state_str = test_cube.to_string()
solver_str = ""

for i in range(54):
    solver_str += number_to_face[int(list(state_str)[state_conversion[i]])]

print(solver_str)

result = sv.solve(solver_str)
print(result[-5:-3])

# Test h(a) <= h(b) + d(a, b) for all a, b in N

LFRBULDUBUFBDRBFRBRBLFFRDRUFFLDDURBDFLBLLDULRUUDRBULDF
depth 14 done in 0.19 s, 66672 nodes generated, about 356379 nodes/s
depth 15 done in 2.88 s, 1002285 nodes generated, about 348317 nodes/s
depth 16 done in 43.76 s, 14928120 nodes generated, about 341104 nodes/s
depth 17 done in 647.46 s, 216485568 nodes generated, about 334364 nodes/s


KeyboardInterrupt: 

In [ ]:
# Test h(n, g) <= d(n, g) for all n in N
error = []
test_admissible_heuristic()

# Test Fail for L D F => 3.001825
# w of 1.5 works

test_admissible_heuristic()